# Deutsche Bahn data scraping

## Basics

In [4]:
import requests
import pandas as pd
import csv

In [3]:
url_parking = 'https://apis.deutschebahn.com/db-api-marketplace/apis/parking-information/db-bahnpark/v2/'
url_ris_stations = 'https://apis.deutschebahn.com/db-api-marketplace/apis/ris-stations/v1/'
client_id = '22a834e6e490f430dc9250bc13e7fba0'
client_secret = '0f294b9c1f1c72f834dbc43cdf33e42d'
limit = 500 # this is the default

In [5]:
headers = {
    'DB-Client-Id': client_id,
    'DB-Api-Key': client_secret,
}

In [4]:
# response = requests.get(url_parking + 'parking-facilities', headers=headers)
# print(response.json())

## Stations

This will return all available stations [Bahnhöfe].

See [here](https://developers.deutschebahn.com/db-api-marketplace/apis/product/ris-stations/api/ris-stations#/RISStations_1134/operation/%2Fstations/get) for documentation.

In [8]:
offset = 0
total = 9999
jsons = []

while (offset < total):
    response = requests.get(url_ris_stations + f'stations?offset={offset}&limit={limit}', headers=headers)
    json = response.json()
    offset += 500
    total = json['total']
    jsons.append(json)

In [23]:
jsons[0]

{'offset': 0,
 'limit': 500,
 'total': 5690,
 'stations': [{'stationID': '1',
   'names': {'DE': {'name': 'Aachen Hbf'}},
   'metropolis': {},
   'address': {'street': 'Bahnhofstr.',
    'houseNumber': '2a',
    'postalCode': '52064',
    'city': 'Aachen',
    'state': 'Nordrhein-Westfalen',
    'country': 'DE'},
   'stationCategory': 'CATEGORY_2',
   'availableTransports': [],
   'availableLocalServices': [],
   'owner': {'name': 'DB S&S',
    'organisationalUnit': {'id': 4,
     'name': 'RB West',
     'nameShort': 'RB West'}},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin',
   'position': {'longitude': 6.091499, 'latitude': 50.7678},
   'validFrom': '2019-01-01T00:00:00Z'},
  {'stationID': '1000',
   'names': {'DE': {'name': 'Burkhardswalde-Maxen'}},
   'metropolis': {},
   'address': {'street': 'Gesundbrunnen',
    'houseNumber': '60c',
    'postalCode': '01809',
    'city': 'Müglitztal-Burkhardswalde',
    'state': 'Sachsen',
    'country': 'DE'},
   'stationCategory': 'CA

In [25]:
data_list = []

for json in jsons:
    for entry in json['stations']:
        data_list.append(
            {
                "id": entry['stationID'],
                "name": entry['names'].get('DE', {}).get('name', ''),
                "metropolis": entry.get('metropolis', ''),
                "street": entry['address'].get('street', ''),
                "houseNumber": entry['address'].get('houseNumber', ''),
                "postalCode": entry['address'].get('postalCode', ''),
                "city": entry['address'].get('city', ''),
                "state": entry['address'].get('state', ''),
                "country": entry['address'].get('country', ''),
                "stationCategory": entry.get('stationCategory', ''),
                "owner": entry['owner'].get('name', ''),
                "organisationalUnit": entry['owner'].get('organisationalUnit', {}).get('name', ''),
                "countryCode": entry['countryCode'],
                "latitude": entry.get('position', {}).get('latitude', ''),
                "longitude": entry.get('position', {}).get('longitude', ''),
                "timeZone": entry['timeZone']
            }
        )


In [20]:
len(data_list)

5690

In [21]:
csv_file_path = 'data.csv'

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data_list[0].keys())
    
    # Write header
    writer.writeheader()
    
    # Write data rows
    writer.writerows(data_list)

## Stop-places

In [ ]:
offset = 0
total = 9999
jsons = []
keyType = ''

while (offset < total):
    response = requests.get(url_ris_stations + f'stop-places?keyType={keyType}abc&key={key}', headers=headers)
    json = response.json()
    offset += 500
    total = json['total']
    jsons.append(json)

In [ ]:
data_list = []

for json in jsons:
    for entry in json['stations']:
        data_list.append(
            {
                "id": entry['stationID'],
                "name": entry['names'].get('DE', {}).get('name', ''),
                #"metropolis": entry['metropolis'],
                "street": entry['address'].get('street', ''),
                "houseNumber": entry['address'].get('houseNumber', ''),
                "postalCode": entry['address'].get('postalCode', ''),
                "city": entry['address'].get('city', ''),
                "state": entry['address'].get('state', ''),
                "country": entry['address'].get('country', ''),
                "stationCategory": entry.get('stationCategory', ''),
                "owner": entry['owner'].get('name', ''),
                "countryCode": entry['countryCode'],
                "latitude": entry.get('position', {}).get('latitude', ''),
                "longitude": entry.get('position', {}).get('longitude', ''),
                "timeZone": entry['timeZone']
            }
        )